# Modeling

## Import Packages

In [1]:
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## Load data

In [2]:
# Load train data
with open('../data/final/train.pickle', 'rb') as train:
    train_data = pickle.load(train)
    
# Load test data
with open('../data/final/test.pickle', 'rb') as test:
    test_data = pickle.load(test)
    
# Load val data
with open('../data/final/val.pickle', 'rb') as val:
    val_data = pickle.load(val)

In [3]:
X_train, y_train = train_data
X_test, y_test = test_data
X_val, y_val = val_data

## Modeling - Baseline

### Create Model

In [4]:
baseline = Sequential([
    # Flatten
    Flatten(input_shape=(150, 150, 3)),
    # 128 Neuron (Hidden layer)
    Dense(128, activation='relu'),
    # 32,Neuron (Hidden layer)
    Dense(32, activation='relu'),
    # 1 Output neuron
    Dense(1, activation='sigmoid')
])

baseline.summary()

Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 67500)             0         
                                                                 
 dense (Dense)               (None, 128)               8640128   
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8,644,289
Trainable params: 8,644,289
Non-trainable params: 0
_________________________________________________________________


2022-05-13 16:20:38.100821: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-13 16:20:38.101061: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
baseline.compile(loss='binary_crossentropy',
                 optimizer=Adam(learning_rate=0.001),
                 metrics=['accuracy'])

### Model Training

In [6]:
history = baseline.fit(X_train,
                       y_train,
                       batch_size=64,
                       epochs=10,
                       verbose=1,
                       validation_data=(X_val, y_val))

Epoch 1/10


2022-05-13 16:20:38.932839: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-13 16:20:39.135142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - 4s 73ms/step - loss: 2.0563 - accuracy: 0.7295 - val_loss: 0.5387 - val_accuracy: 0.8125
Epoch 2/10
 1/38 [..............................] - ETA: 2s - loss: 0.2950 - accuracy: 0.9219

2022-05-13 16:20:42.758125: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - 3s 66ms/step - loss: 0.2682 - accuracy: 0.9185 - val_loss: 0.4018 - val_accuracy: 0.8750
Epoch 3/10
38/38 [==============================] - 3s 66ms/step - loss: 0.1298 - accuracy: 0.9537 - val_loss: 0.2370 - val_accuracy: 0.9375
Epoch 4/10
38/38 [==============================] - 2s 66ms/step - loss: 0.4896 - accuracy: 0.8722 - val_loss: 3.7614 - val_accuracy: 0.5625
Epoch 5/10
38/38 [==============================] - 2s 65ms/step - loss: 0.5021 - accuracy: 0.8908 - val_loss: 1.2307 - val_accuracy: 0.7500
Epoch 6/10
38/38 [==============================] - 2s 65ms/step - loss: 0.2237 - accuracy: 0.9338 - val_loss: 0.1424 - val_accuracy: 0.9375
Epoch 7/10
38/38 [==============================] - 2s 65ms/step - loss: 0.1424 - accuracy: 0.9520 - val_loss: 0.2183 - val_accuracy: 0.9375
Epoch 8/10
38/38 [==============================] - 2s 66ms/step - loss: 0.1271 - accuracy: 0.9582 - val_loss: 0.2020 - val_accuracy: 0.8750
Epoch 9/10
38/38 [======

### Test Accuray

In [7]:
baseline.evaluate(X_test, y_test)

20/20 [==============================] - 1s 47ms/step - loss: 0.5693 - accuracy: 0.8269


[0.5693485736846924, 0.8269230723381042]

# Modeling - Tuning

### Create Tuned Model

In [58]:
model = Sequential([
    # The first convolution
    Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPool2D(2, 2),
    # The second convolution
    Conv2D(128, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
     # The Third convolution
    Conv2D(128, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    # Flatten
    Flatten(),
    # 8 Neuron (Hidden layer)
    Dense(128, activation='relu', kernel_regularizer=regularizers.L2(0.005)),
    # 1 Output neuron
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_37 (Conv2D)          (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 74, 74, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_38 (Conv2D)          (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 36, 36, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_39 (Conv2D)          (None, 34, 34, 128)       147584    
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 17, 17, 128)    

In [59]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.00001),
              metrics=['accuracy'])

### Train Tuned Model

In [60]:
es = EarlyStopping(patience=5)
mc = ModelCheckpoint('../models/checkpoint/tuned.h5', save_best_only=True, monitor='val_loss')

In [61]:
history = model.fit(X_train,
                    y_train,
                    batch_size=64,
                    epochs=30,
                    verbose=1,
                    validation_data=(X_val, y_val),
                    callbacks=[es, mc])

Epoch 1/30


2022-05-13 17:08:41.485163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - ETA: 0s - loss: 1.9125 - accuracy: 0.6199

2022-05-13 17:08:57.460985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - 16s 420ms/step - loss: 1.9125 - accuracy: 0.6199 - val_loss: 1.8660 - val_accuracy: 0.6250
Epoch 2/30
38/38 [==============================] - 16s 411ms/step - loss: 1.7931 - accuracy: 0.7432 - val_loss: 1.7600 - val_accuracy: 0.7500
Epoch 3/30
38/38 [==============================] - 16s 414ms/step - loss: 1.6588 - accuracy: 0.8652 - val_loss: 1.6480 - val_accuracy: 0.7500
Epoch 4/30
38/38 [==============================] - 16s 413ms/step - loss: 1.5058 - accuracy: 0.8912 - val_loss: 1.5453 - val_accuracy: 0.7500
Epoch 5/30
38/38 [==============================] - 16s 414ms/step - loss: 1.3555 - accuracy: 0.8978 - val_loss: 1.4379 - val_accuracy: 0.7500
Epoch 6/30
38/38 [==============================] - 16s 413ms/step - loss: 1.2306 - accuracy: 0.9049 - val_loss: 1.3618 - val_accuracy: 0.7500
Epoch 7/30
38/38 [==============================] - 16s 413ms/step - loss: 1.1320 - accuracy: 0.9189 - val_loss: 1.2916 - val_accuracy: 0.7500
Epoch 8/30

### Test Accuracy

In [62]:
model.load_weights('../models/checkpoint/tuned.h5')
model.evaluate(X_test, y_test)

20/20 [==============================] - 2s 79ms/step - loss: 0.6992 - accuracy: 0.8638


[0.6991806626319885, 0.8637820482254028]

In [63]:
model.save('../models/final_model.h5')